> **Note:** The notebook assumes that the code is being executed in Kaggle. If this is not the case, you will need to make some minor changes.

> This notebook deals only with fine-tuning the model. After fine-tuning, you need to merge the LoRA layer to the base model. This is covered in the notebook: [Merging_Adapter_with_Base_Model.ipynb](https://github.com/BitwiseBrains/RagOptimize/blob/main/fine_tuning/Merging_Adapter_with_Base_Model.ipynb).

# Preparation


We need to install some libraries to use in this notebook. Some of other libraries are already installed in the Kaggle environment, but we need to upgrade them to the latest version.


In [ ]:
!pip install -U -q datasets bitsandbytes accelerate torch peft==0.6.0

In [ ]:
from datasets import load_dataset, Dataset
import torch
from torch.optim import AdamW
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    MistralForCausalLM,
)
from peft import (
    PeftModel,
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
)
import gc
import wandb
import yaml
import os
import time

from huggingface_hub import HfApi, CommitOperationAdd, login, hf_hub_download

# Config and Logins


## Configuration

We will be using a YAML file to store the configuration. A configuration file is a good practice to keep the code clean and easy to maintain. We will use the `pyyaml` library to read the configuration file.


Here are details about the configuration file:

- `hf_repo_id`: The ID of the Hugging Face repository where the model is stored.
- `start_batch_number`: The batch number to start training from. (Since we can not train the complete dataset in one go, as Kaggle has a time limit for the training, we will train the model in batches.)
- `end_batch_number`: The batch number to end training at.
- `base_model_id`: The ID of the base model to be fine-tuned.
- `load_pretrained`: Whether to load a pretrained model. When starting from scratch, this should be set to `False` after that it should be set to `True`. Setting this to `True` will load the model from the base model from `base_model_id` and the head and adapter from `hf_repo_id` and will merge them to create the complete model.
- `head_file_name`: The name of the file containing the model head.
- `dataset_id`: The ID of the dataset to be used for training.
- `quantization_config`: Configuration for model quantization.
  - `load_in_4bit`: Whether to load the model in 4-bit precision.
  - `bnb_4bit_quant_type`: The type of 4-bit quantization to use.
  - `bnb_4bit_use_double_quant`: Whether to use double quantization.
  - `bnb_4bit_compute_dtype`: The data type to use for computation.
- `lora_config`: Configuration for LoRA (Long Range Attention).
  - `r`: The reduction factor for the attention mechanism.
  - `lora_alpha`: The alpha parameter for LoRA.
  - `lora_dropout`: The dropout rate for LoRA.
  - `bias`: The bias for the attention mechanism.
  - `task_type`: The type of task to be performed.
  - `target_modules`: The modules to be targeted by LoRA.
- `num_warmup_steps`: The number of warmup steps for the optimizer.
- `epochs`: The number of epochs to train for.
- `max_iter_per_epoch`: The maximum number of iterations per epoch. This can be helpful in debugging and testing.
- `max_steps`: The maximum number of steps for the training.
- `log_interval`: The interval at which to log training information.
- `wandb`: Whether to use Weights & Biases for logging.
- `project`: The name of the Weights & Biases project.
- `wandb_name`: This will be used along with the `start_batch_number` and `end_batch_number` to create a name for the run.
- `notes`: Notes about the training run.
- `lr`: The starting learning rate for the optimizer.
- `accumulation_steps`: The number of steps to accumulate gradients before updating the model parameters. We can set it to $\gt 1$ for gradient accumulation.
- `batch_size`: The size of the batches for training.
- `max_length`: The maximum length of the sequences for training. This will decide what is the maximum number of tokens in the input sequences.
- `model_save_root_dir`: The directory where the trained models will be saved.
- `push_to_hub`: Whether to push the trained model to the Hugging Face Model Hub.
- `push_to_hub_frequency`: The frequency at which to push the model to the hub.
- `max_hours`: The maximum number of hours to train for. This is useful to avoid the training to run for too long and get killed by Kaggle.


In [ ]:
config = """---
hf_repo_id: hari31416/Mistral_Finance_Finetuning
start_batch_number: 0
end_batch_number: 1000
base_model_id: mistralai/Mistral-7B-Instruct-v0.1
load_pretrained: False
head_file_name: mistral_head.pt
dataset_id: gbharti/finance-alpaca
quantization_config:
  load_in_4bit: True
  bnb_4bit_quant_type: nf4
  bnb_4bit_use_double_quant: True
  bnb_4bit_compute_dtype: bfloat16
lora_config:
  r: 16
  lora_alpha: 4
  lora_dropout: 0.05
  bias: none
  task_type: CAUSAL_LM
  target_modules:
    - o_proj
    - v_proj
    - k_proj
    - q_proj
num_warmup_steps: 0
epochs: 1
max_iter_per_epoch:
max_steps:
log_interval: 1
wandb: True
project: RAGOptimize
wandb_name: fine_tune_multi_epoch
notes: RAGOptimize Training multiple epochs
lr: 0.0001
accumulation_steps: 1
batch_size: 4
max_length: 1024
model_save_root_dir: /kaggle/working/models
push_to_hub: True
push_to_hub_frequency: 100
max_hours: 11.7
"""
config = yaml.safe_load(config)
# Create the model save root directory so that we can save the model
os.makedirs(config["model_save_root_dir"], exist_ok=True)

## Login to W&B and HF

We will be saving the training log to W&B. We will use the `wandb` library to log the training process. First, we need to login to W&B to use the service. One way to do this is to use the `wandb.login()` function. Since this function creates a file at `/root/.netrc` with the content:

```text
machine api.wandb.ai
login user
password <WANDB_API_KEY>
```

we can create this file manually and use the `wandb` library without calling the `wandb.login()` function.


We will also be pushing the trained model at various instances to Hugging Face. For this, we need to login to Hugging Face. We will be using the `login` method from `huggingface_hub` library to login to Hugging Face.


In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
if config["wandb"]:
    WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

    text = f"""machine api.wandb.ai
      login user
      password {WANDB_API_KEY}
    """
    # wandb saves credentials at /root/.netrc
    with open("/root/.netrc", "w") as f:
        f.write(text)
        name = f"""{config["wandb_name"]}_{config["start_batch_number"]}_{config["end_batch_number"]}"""
    print(f"Creating WANDB run with name: {name}")
    wandb.init(
        project=config["project"], name=name, notes=config["notes"], config=config
    )

if config["push_to_hub"]:
    HUGGING_FACE_API_KEY = user_secrets.get_secret("HUGGING_FACE_API_KEY")
    login(HUGGING_FACE_API_KEY)

api = HfApi()

# Loading the Model and Tokenizer


There are a couple of steps involved to load the model. These are:

- Since we can not load the whole model in memory in full precision, we will load the model in 4-bit precision. For this, we need to create the quantization configuration and load the model in 4-bit precision.
- We are using PEFT and LoRA in the model and hence need to create the configuration for LoRA.
- Load the base model.
- If we are not starting from scratch, we will load the pretrained model adapter and head from Hugging Face and merge it with the base model to create the complete model. We make sure that the head weights are initialized with the pretrained weights and they are different from the head weights of the base model.
- If we are starting from scratch, we will create PEFT model using the LoRA configuration and the base model.
- We make the head weights trainable.


In [ ]:
# use bfloat16 or float16 depending on the config
bnb_4bit_compute_dtype = (
    torch.bfloat16
    if config["quantization_config"]["bnb_4bit_compute_dtype"] == "bfloat16"
    else torch.float16
)
config["quantization_config"].update({"bnb_4bit_compute_dtype": bnb_4bit_compute_dtype})

# load the quantization and lora config
quantization_config = BitsAndBytesConfig(**config["quantization_config"])
lora_config = LoraConfig(**config["lora_config"])

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(config["base_model_id"])
tokenizer.pad_token_id = tokenizer.eos_token_id

# load the base model
model = MistralForCausalLM.from_pretrained(
    config["base_model_id"],
    quantization_config=quantization_config,
)
# save the original head weights to check if the head weights are updated
og_head = model.lm_head.state_dict()["weight"].to(
    "cpu"
)  # move to cpu to avoid getting the weights updated

# enable gradient checkpointing and prepare the model for kbit training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

if config["load_pretrained"]:
    # if loading pretrained, load the head weights
    print("Loading pretrained PEFT Model and the head")
    head_file_path = hf_hub_download(
        config["hf_repo_id"], config["head_file_name"], local_dir="."
    )
    lm_head_state_dict = torch.load(head_file_path)
    model.lm_head.load_state_dict(lm_head_state_dict)

    # load the adapter to the base model
    model = PeftModel.from_pretrained(model, config["hf_repo_id"], is_trainable=True)

    # if loading pretrained, make sure that the weights of the head are different
    new_head = model.lm_head.state_dict()["weight"].to("cpu")
    if torch.equal(new_head, og_head):
        raise ValueError("Head weights are the same!")
    print("Head weights are different.")
    # delete the head weights for memory
    del lm_head_state_dict, new_head
else:
    # If not loading pretrained, create the PEFT model using the LORA config
    print("Creating the PEFT model using LORA config")
    model = get_peft_model(model, lora_config)

# delete the original head weights for memory
del og_head
# make the head trainable
model.base_model.model.lm_head.weight.requires_grad_()

# print the trainable parameters
model.print_trainable_parameters()

In [ ]:
# make sure that the head weights are trainable
assert model.base_model.model.lm_head.weight.requires_grad

# The Dataset


## Load the Dataset

Next step is to load the dataset. We will be using the `load_dataset` function from the `datasets` library to load the dataset. The original dataset has columns like `"text", "instruction", "output", "input"`. Since we are doing a fine-tuning, we will use these columns to create the input of the model. The output of the model will be created automatically when creating `DataLoader` by using the `DataCollatorForLanguageModeling`. As we are using instruct variant of Mistral as the base model, the input must be in the format of `<INST>{user_content}</INST>{output}`. For this, we will create a function to create the input from the dataset in the required format.

In [ ]:
dataset = load_dataset(config["dataset_id"])
dataset = dataset["train"]
TOTAL_SAMPLES = len(dataset)
TOTAL_BATCHES = TOTAL_SAMPLES // config["batch_size"]
print(
    f"Total number of samples: {TOTAL_SAMPLES}\nTotal Number of Batches: {TOTAL_BATCHES}"
)


def format_input_text(text, verbose=False):
    """Formats the input text to the format required by the model.
    
    Parameters
    ----------
    text : dict
        The input text dictionary containing the input, instruction, text and output.
    verbose : bool, optional
        Whether to print the formatted message, by default False
    """
    input_ = text["input"]
    instruction = text["instruction"]
    text_ = text["text"]
    output = text["output"]
    user_content = ""
    if input_:
        user_content += f"{input_}\n"
    if instruction:
        user_content += f"{instruction}\n"
    if text_:
        user_content += f"{text_}\n"
    user_content = user_content.strip()
    message = f"<s><INST>{user_content}</INST>{output}"
    if verbose:
        print(message)
    return {"message": message}

# use the format_input_text function to format the input text
dataset = dataset.map(format_input_text)
# remove the columns that are not required
dataset = dataset.remove_columns(["text", "instruction", "output", "input"])

## Filter The Dataset

We can not train the model on complete dataset in one go, as Kaggle has a time limit for the training. Hence, we will train the model in batches. We will use the `start_batch_number` and `end_batch_number` from the configuration to decide which batches to train. Using these parameters and the `batch_size`, we will decide which part of the dataset to use for training. Then using the `select` method from the `datasets` library, we will select the required part of the dataset.

In [ ]:
dataset_start_idx = config["start_batch_number"] * config["batch_size"]
dataset_end_idx = config["end_batch_number"] * config["batch_size"]
print(f"Splitting from {dataset_start_idx} to {dataset_end_idx}")
data = dataset.select(range(dataset_start_idx, dataset_end_idx))
print(f"Number of samples to be trained on: {len(data['message'])}")

## Create a Custom `Dataset`

Now, we will create a `Dataset` class that implements the `__getitem__` and `__len__` methods. The `__getitem__` method will create the tokens from the message to be trained. This class will be used to create the `DataLoader` for training. We will use the `DataCollatorForLanguageModeling` to create the `DataLoader`. The collator class will take care of creating the input and output for the model from the dataset.

In [ ]:
class FinanceDataset(Dataset):
    """A custom dataset class for the finance dataset"""
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        """Creates the input tokens for the model"""
        d = self.dataset[idx]
        message = d["message"]
        tokens = self.tokenizer(
            message,
            return_tensors="pt",
            max_length=config["max_length"],
            truncation=True,
            padding=True,
        )
        return tokens

# The data collator for the language modeling task. This will make sure that the data is in correct format for the model
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_dataset = FinanceDataset(data, tokenizer)
# Create the dataloader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    collate_fn=data_collator,
)

# Training


## The Native `Trainer` Class in PyTorch

Now that all the preparation steps are done, we are ready to train the model. For this, we will create a custom `Trainer` class that implements the training and validation loop along with some other features, like logging and saving the model, making sure that the model is pushed to Hugging Face at regular intervals, etc.

In [ ]:
class Trainer:
    """A class to train the model. This class can be used to train the model for multiple epochs."""

    def __init__(
        self,
        model: torch.nn.Module,
        train_dataloader: DataLoader,
        test_dataloader: DataLoader = None,
        max_iter_per_epoch=None,
        optimizer=None,
        scheduler=None,
        wandb_run=None,
        log_interval: int = 100,
        max_hours=None,
        push_to_hub_frequency=None,
    ) -> None:
        """Initializes the Trainer class

        Parameters
        ----------
        model : torch.nn.Module
            The model to be trained
        train_dataloader : DataLoader
            The dataloader for the training dataset
        test_dataloader : DataLoader, optional
            The dataloader for the test dataset, by default None. If None, the validation step will be skipped
        max_iter_per_epoch : _type_, optional
            The maximum number of iterations per epoch, by default None
        optimizer : _type_, optional
            The optimizer to be used for training, by default None
        scheduler : _type_, optional
            The scheduler to be used for training, by default None
        wandb_run : _type_, optional
            The wandb run object, by default None. If None, the model will not be logged to wandb
        log_interval : int, optional
            The interval at which the logs should be printed, by default 100
        max_hours : _type_, optional
            The maximum number of hours to train, by default None. If None, the training will not stop based on time
        push_to_hub_frequency : _type_, optional
            The frequency at which the model should be pushed to hub, by default None. If None, the model will not be pushed to hub
        """
        self.model = model
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.max_iter_per_epoch = max_iter_per_epoch
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if optimizer:
            self.optimizer = optimizer
        else:
            self.optimizer = AdamW(self.model.parameters(), lr=1e-4)

        self.scheduler = scheduler
        self.step = 0
        self.batch_step = 0
        self.wandb_run = wandb_run
        self.log_interval = log_interval
        self.tic = time.time()
        self.max_hours = max_hours
        self.push_to_hub_frequency = push_to_hub_frequency
        if wandb_run is not None:
            self.wandb_run.watch(self.model.lm_head, log_freq=100)

    def check_break_loop(self):
        """Checks if the training should be stopped based on the time"""
        if self.max_hours is None:
            return False
        toc = time.time()
        seconds = toc - self.tic
        break_ = seconds / 3600 >= self.max_hours
        if break_:
            print(f"Stopping training as {seconds/3600} has passed")
        return break_

    def calculate_eta(self):
        """Calculates the estimated time for the training to complete in hours"""
        toc = time.time()
        seconds = toc - self.tic
        curent_step = self.step
        max_iter_per_epoch = self.max_iter_per_epoch or len(self.train_dataloader)
        max_steps = max_iter_per_epoch * config["epochs"]
        eta = (max_steps - curent_step) * seconds / curent_step
        eta = eta / 3600
        return eta

    def trainings_step(self, batch):
        """Implements the training step for the model"""
        inputs = {k: v.to(self.device) for k, v in batch.items()}
        outputs = self.model(**inputs)
        loss = outputs.loss
        del inputs, outputs
        gc.collect()
        return loss

    def validation_step(self, batch):
        """Implements the validation step for the model"""
        with torch.no_grad():
            inputs = {k: v.to(self.device) for k, v in batch.items()}
            outputs = self.model(**inputs)
            loss = outputs.loss
            del inputs, outputs
            gc.collect()
        return loss

    def train(self, epochs, max_steps=None, verbose=False, accumulation_steps=1):
        """The main training loop for the model

        Parameters
        ----------
        epochs : int
            The number of epochs to train the model
        max_steps : int, optional
            The maximum number of steps to train, by default None
        verbose : bool, optional
            Whether to print the logs, by default False
        accumulation_steps : int, optional
            The number of steps to accumulate the gradients, by default 1
        """
        history = {"train_loss": [], "val_loss": []}
        max_iter_per_epoch = self.max_iter_per_epoch or len(self.train_dataloader)
        for epoch in range(epochs):
            train_loss = 0
            # create a progress bar for the training
            t = tqdm(self.train_dataloader, total=max_iter_per_epoch)
            for batch in t:
                # batch_max_length will be logged to wandb and is only for debug purposes
                batch_max_length = batch["input_ids"].shape[
                    1
                ]  # only for debug purposes
                self.model.train()
                self.step += 1
                self.batch_step += 1
                loss = self.trainings_step(batch)

                # clear the memory after each batch
                del batch
                gc.collect()
                torch.cuda.empty_cache()

                loss = loss / accumulation_steps
                loss.backward()
                # implement gradient accumulation
                if ((self.batch_step + 1) % accumulation_steps == 0) or (
                    self.batch_step + 1 == len(self.train_dataloader)
                ):
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    if self.scheduler:
                        self.scheduler.step()

                batch_loss = loss.item()
                train_loss += batch_loss
                if self.check_break_loop() or (max_steps and self.step >= max_steps):
                    # if the training should be stopped, save the model and return the history
                    self.save_model(
                        f"{config['model_save_root_dir']}/model_e{epoch+1}.pt"
                    )
                    return history

                self.model.eval()
                train_loss_ = train_loss / self.batch_step
                # add the loss and lr to the progress bar
                lr = self.optimizer.param_groups[0]["lr"]
                t.set_postfix({"loss": train_loss_, "lr": lr})
                if self.step % self.log_interval == 0:
                    if self.wandb_run:
                        # log the metrics and stuff to wandb
                        eta = self.calculate_eta()
                        self.wandb_run.log(
                            {
                                "train_loss": train_loss_,
                                "batch_loss": batch_loss,  # only for debug purposes
                                "lr": lr,
                                "cur_step": self.step,
                                "abs_step": self.step + config["start_batch_number"],
                                "etr": eta,  # to get an idea about how long the run will take
                                "batch_max_length": batch_max_length,  # only for debug purposes
                            }
                        )
                    history["train_loss"].append(train_loss_)
                    if verbose:
                        print(f"Step {self.step} Loss: {train_loss_}")
                if (
                    self.max_iter_per_epoch is not None
                    and self.max_iter_per_epoch < self.batch_step
                ):
                    # complete the epoch if the max_iter_per_epoch is reached
                    break

                if (
                    self.push_to_hub_frequency
                    and self.step % self.push_to_hub_frequency == 0
                ):
                    # push the model to hub based on the frequency
                    self.push_to_hub()

            train_loss_ = train_loss / self.batch_step
            print(f"Epoch {epoch+1:2d} || Train Loss: {train_loss_:.4f}")
            self.batch_step = 0

            if self.test_dataloader:
                # if the test dataloader is provided, run the validation step
                val_loss = 0
                self.model.eval()
                for batch in tqdm(self.test_dataloader):
                    loss = self.validation_step(batch)
                    val_loss += loss.item()
                val_loss_ = val_loss / len(self.test_dataloader)
                if self.wandb_run:
                    self.wandb_run.log({"val_loss": val_loss_})
                history["val_loss"].append(val_loss_)
                print(f"Epoch {epoch+1:2d} || Validation Loss: {val_loss_:.4f}")
        return history

    def save_model(self, path):
        """Saves the model to the given path"""
        print(f"Saving model to: {path}")
        torch.save(self.model.state_dict(), path)

    def clear_memory(self):
        """Clears the memory"""
        del self.model
        print("Clearing memory")
        gc.collect()
        torch.cuda.empty_cache()

    def push_to_hub(self):
        """Pushes the adapter and the head to the hub"""

        # Push the adapter to the hub
        start_step = config["start_batch_number"]
        end_step = start_step + self.step
        commit_message = f"Trained model from {start_step} to {end_step} steps"
        print(f"Pushing the model to hub with commit: {commit_message}")
        self.model.push_to_hub(config["hf_repo_id"], commit_message=commit_message)

        # save the dict containing the model head state to hub using the Hugging Face API
        head = self.model.lm_head
        file_name = config["head_file_name"]
        torch.save(head.state_dict(), file_name)
        operations = [
            CommitOperationAdd(path_in_repo=file_name, path_or_fileobj=file_name)
        ]
        commit_message = f"Adding head to model from {start_step} to {end_step} steps"
        print(f"Pushing the head to hub with commit: {commit_message}")
        api.create_commit(
            config["hf_repo_id"],
            operations=operations,
            commit_message=commit_message,
        )

Before we start training the model, we will clear the CUDA cache to make sure that we have enough memory to train the model.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

## Training The Model

The final step involves creating:

- The optimizer. We will be using the `AdamW` optimizer.
- The scheduler. We will be using the `get_cosine_schedule_with_warmup` scheduler.
- We will decide whether we want to log to wandb or not. If we want to log to wandb, we will create a `wandb_run` and log the configuration to it.
- Finally, we will wrap the whole `trainer.train` method in a `try-except` block to make sure that even if the training fails, we are saving the latest model and pushing it to Hugging Face.

In [ ]:
epochs = config["epochs"]
max_iter_per_epoch = config["max_iter_per_epoch"] or len(train_dataloader)
log_interval = config["log_interval"]
num_warmup_steps = config["num_warmup_steps"]
max_steps = config["max_steps"]
max_hours = config["max_hours"]
accumulation_steps = config["accumulation_steps"]
push_to_hub_frequency = config["push_to_hub_frequency"]

if config["wandb"]:
    wandb_run = wandb
else:
    wandb_run = None
optimizer = AdamW(model.parameters(), lr=config["lr"])
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=(TOTAL_BATCHES - config["start_batch_number"]) * epochs,
)
trainer = Trainer(
    model,
    train_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    wandb_run=wandb_run,
    log_interval=log_interval,
    max_iter_per_epoch=max_iter_per_epoch,
    max_hours=max_hours,
    push_to_hub_frequency=push_to_hub_frequency,
)
try:
    history = trainer.train(
        epochs,
        max_steps=max_steps,
        verbose=False,
        accumulation_steps=accumulation_steps,
    )
except Exception as e:
    print(e)
finally:
    trainer.push_to_hub()
    trainer.clear_memory()
    wandb.finish()

# Changelog


**Version 1:**

Added the starting code and trained from step 40 to 1840. (Batch 0 to 40 were trained while experimenting with the code.)

**Version 2:**

Trained from step 1840 to 5000.

**Version 3:**

Added ETA to the wandb log and trained from step 5000 to 5100 to make sure the code is working as intended

**Version 4:**

Training from steps 5100 to 9000. This is about 4000 batches, hopefully the whole training will be done within 12 hours. Also, using the starting learning rate from the last learning rate of the version 2. This will be followed in next versions.

**Version 5:**

Training from steps 9000 to 13500, as the last run took only 5.5 hour. If this run is completed, only one extra run will be required for the whole finetuning. Using somewhat higher learning rate than the learning rate from the last batch. Changed `eta` to `etr (h)`. Also, calling `wandb_run.log` only one time to avoid the step count issue.

**Version 6:**

Training from steps 13500 to 17225 (final batch being 17228). This will be final training iteration. Using somewhat higher learning rate than the learning rate from the last batch as before.

**Version 7:**

Changed the `hf_repo_id` from`hari31416/RAGOptimize` to `hari31416/RAGOptimize_Adapter`.

**Version 8:**

The response from the model was not coming out to be as intended. For this, made the following changes:

- Made the head of the model trainable
- Changed the `lora_alpha` to 4
- Saving the model head to the hub and loading the latest head when required
- Logging the batch loss to wandb

Using these changes, started training the model from step 0 to 2000.

**Version 9:**

Fix issue with `file_name` variable name.

**Version 10:**

Trained from step 2000 to 5000.

**Version 11:**

Fixed issue with saving model head (the model head which was saved was the original head). Watching the model head with wandb. Added code for checking the head weights before starting the training.

**Version 12:**

- Moving the head weights to CPU before matching them. Otherwise, the original weight will get updated after the new weight is loaded, which sometimes result in raising the `ValueError` even though the weights have changed.
- Deleting th extra weights after the new head is set.
- Calling `trainer.push_to_hub()` before `trainer.clear_memory()` to avoid error raised since calling `trainer.clear_memory()` will result in deletion of model.

**Version 13:**

- Training from 1000 to 5000 batches
- Stripping the the `user_content` to avoid the extra new line and starting the model response from the same line.
- Starting lr set to $0.00009965668$ (equal to the last learning rate in previous batch).
- Logging the max length in the batch for debugging purposes.

**Version 14:**

Training from 4280 to 7500 batch. (The training till 5000 batches was not successful in the previous run.) Starting lr set to $0.0000899500958$.

**Version 15:**

Setting `verbose` to `False` to avoid printing out the loss at each batch. Wandb is already tracking everything. Other parameters same as `Version 14`.

**Version 16:**

Training from 7500 to 12500 batch.

**Version 17:**

Training from 12300 to 15300 batch and for 6 hours only.

**Version 21:**

Traning from 0 to 1000 batch for two epochs. Updated the `num_training_steps` argument in scheduler to take into account the situation when training needs to be done for more than one epoch. Fixed the run name from `fine_tune_mutli_epoch` to `fine_tune_multi_epoch`.

**Version 22:**

Adding notes and comments in the code.
